In [18]:
import os
import numpy as np
import pandas as pd


In [19]:
models = os.listdir(f"Modules")

In [20]:
os.system("vivado -mode batch -source tcl_create.tcl")   #creating new project

0

In [21]:
for mod in models:
    os.system(f"vivado -mode batch -source  tcl_add.tcl -tclargs {mod}") # adding the modules to the project

In [22]:
os.mkdir("results")

In [23]:
data_files = [] #only append implemented files in the data files list

for filename in models:
    os.mkdir("results/"+filename[:-2])
    os.system("vivado -mode batch -source  tcl_run.tcl -tclargs {}".format(filename))
    synth_file = open("synth_status", "r")
    synth_status = synth_file.read().split()[-1]
    if synth_status == "ERROR":
        print(f"Synthesis failed for {filename}")
        continue
    impl_file = open("impl_status", "r")
    impl_status = impl_file.read().split()[-1]
    if impl_status == "ERROR":
        print(f"Implementation failed for {filename}")
        continue
    data_files.append(filename[:-2])
    print("{} reported".format(filename)) 

adder_1bit.v reported
Barrel_shifter.v reported
binary_multiplier.v reported
Implementation failed for convolution.v
FSM_mealy.v reported
FSM_moore.v reported
logic_gate.v reported
shifter.v reported
Synthesis failed for sorter.v
universal_register.v reported


In [24]:
print(data_files)

['adder_1bit', 'Barrel_shifter', 'binary_multiplier', 'FSM_mealy', 'FSM_moore', 'logic_gate', 'shifter', 'universal_register']


In [25]:
file_types = ['power.txt','timing.txt','utilization.txt']

req_data = ['Total On-Chip Power','Data Path Delay','Slice LUTs']

Power = []
Delay = []
LUTs = []

In [26]:
for data in data_files :
    for file in file_types:
        with open("results/{}/{}".format(data,file),'r') as File:
             content = File.read()

        words = content.split('\n')   

        for req_word in words:
            if file_types[0] in file:
                if req_data[0] in req_word:
                    pwr = float(req_word.split()[6])
                    print(pwr)
                    Power.append(pwr)
  

        for req_word in words:
            if file_types[1] in file:
                if req_data[1] in req_word:
                    delay = float(req_word.split()[3][:-2])
                    print(delay)
                    Delay.append(delay)


        for req_word in words:
            if file_types[2] in file:
                if req_data[2] in req_word:
                    luts = float(req_word.split()[4])
                    print(luts) 
                    LUTs.append(luts) 
                    print("_____")    

1.034
5.896
1.0
_____
1.388
6.218
4.0
_____
13.675
10.976
69.0
_____
0.338
4.356
5.0
_____
0.393
4.204
5.0
_____
1.388
6.218
4.0
_____
1.408
4.419
3.0
_____
1.901
4.315
9.0
_____


In [27]:
Final_results = [Power,Delay,LUTs]
row_name = ['Power','Delay','LUTs']

df = pd.DataFrame(Final_results, index = row_name)
df.columns = data_files

df.to_csv('Final_results.csv', index = row_name)
print(df)

       adder_1bit  Barrel_shifter  ...  shifter  universal_register
Power       1.034           1.388  ...    1.408               1.901
Delay       5.896           6.218  ...    4.419               4.315
LUTs        1.000           4.000  ...    3.000               9.000

[3 rows x 8 columns]
